# Detecting distribution shifts

## Setup

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)
from scipy.optimize import fsolve
from scipy.special import erf
from scipy.integrate import quad
from functools import partial
from scipy.stats import binomtest
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from statsmodels.stats.multitest import multipletests
from scipy.stats import percentileofscore


from crepes import ConformalRegressor, ConformalPredictiveSystem

from crepes.fillings import (sigma_variance, 
                            sigma_variance_oob,
                            sigma_knn,
                            binning)

In this notebook we explain how to implement the shift detection algorithm. We start downloading the data and modeling it.

In [62]:
# First we load the dataset
dataset = fetch_openml(name="house_sales",version=3)

X = dataset.data.values.astype(float)
y = dataset.target.values.astype(float)
print('Before normalization range:',y.min(), y.max())

# and also normalize it such that the y values are in the range [0,1]
#y = (np.tanh(np.array([(y[i]-y.min())/(y.max()-y.min()) for i in range(len(y))])*2-1)+1)/2.
#y = np.array([(y[i]-y.min())/(y.max()-y.min()) for i in range(len(y))])
y = np.tanh(y/y.mean())
print('After normalization range:',y.min(), y.max())

/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Before normalization range: 75000.0 7700000.0
After normalization range: 0.13798043023679007 0.9999999999991728


Now we spit the data in the main and shifted data, and form tuples for comparison.

In [63]:
# Then we split the data such that we can later on introduce a distribution shift
X_main, X_shift, y_main, y_shift = X[(4e4 >= X[:, 3])] , X[(4e4 < X[:, 3])] , y[(4e4 >= X[:, 3])] , y[(4e4 < X[:, 3])]

# We want to have an even number of samples, to create tuples
if len(X_shift) % 2 == 1:
    X_shift = X_shift[:-1]
    y_shift = y_shift[:-1]
if len(X_main) % 2 == 1:
    X_main = X_main[:-1]
    y_main = y_main[:-1]

## The input will be two copies of X, one for each house
X_main = X_main.reshape(X_main.shape[0]//2,-1)
X_shift = X_shift.reshape(X_shift.shape[0]//2,-1)
print('After creating tuples',X_main.shape, X_shift.shape)

## We want to estimate the difference between the two y values
y_main = y_main.reshape(y_main.shape[0]//2,-1)
y_main = y_main[:,0] - y_main[:,1]
print(y_main.shape)

## and also in the shifted dataset
y_shift = y_shift.reshape(y_shift.shape[0]//2,-1)
y_shift = y_shift[:,0] - y_shift[:,1]
print(y_shift.shape)

After creating tuples (10202, 42) (604, 42)
(10202,)
(604,)


Finally we create a calibration dataset.

In [64]:
X_train, X_cal, y_train, y_cal = train_test_split(X_main, y_main, test_size=5000)

Now we create the model and train it.

In [65]:
# Then we train a random forest regressor on the training set
random_forest_model = RandomForestRegressor(n_jobs=-1, n_estimators=500) 
random_forest_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=500, n_jobs=-1)

We can also decide how to mix the calibration and test dataset

In [66]:
# Mix the calibration and the shifted data maintaining their sizes
calsize = len(X_cal)//3 # a percentage of the calibration and shifted sets will be mixed up
shiftsize = len(X_shift)//3

np.random.shuffle(X_cal)
np.random.shuffle(X_shift)
np.random.shuffle(y_cal)
np.random.shuffle(y_shift)

X_cal_shift, X_cal_remain, X_shift_remain = np.concatenate((X_cal[:calsize], X_shift[:shiftsize]), axis=0), X_cal[calsize:], X_shift[shiftsize:]
y_cal_shift, y_cal_remain, y_shift_remain = np.concatenate((y_cal[:calsize], y_shift[:shiftsize]), axis=0), y_cal[calsize:], y_shift[shiftsize:]

X_cal_new, X_shift_new, y_cal_new, y_shift_new = train_test_split(X_cal_shift, y_cal_shift, test_size=len(X_cal_shift)//2)

# mix the new and remain sets
np.random.shuffle(np.concatenate((X_cal_new, X_cal_remain), axis=0))
np.random.shuffle(np.concatenate((X_shift_new, X_shift_remain), axis=0))
np.random.shuffle(np.concatenate((y_cal_new, y_cal_remain), axis=0))
np.random.shuffle(np.concatenate((y_shift_new, y_shift_remain), axis=0))


## Conformal model

We will target a success probability $1-\delta = 0.95$. Then we train the conformal model on the residuals on the predictions of the random forest.

In [67]:
delta = 0.05
epsilon = 0.05

cr_std = ConformalRegressor()
y_hat_cal = random_forest_model.predict(X_cal)
residuals_cal = y_cal - y_hat_cal
# and fit it to the residuals
cr_std.fit(residuals=residuals_cal)

ConformalRegressor(fitted=True, normalized=False, mondrian=False)

We can now make prediction on a given test set.

In [68]:
# Using such model we can now predict the residuals on the test set
y_hat_shift = random_forest_model.predict(X_shift)
intervals_std = cr_std.predict(y_hat=y_hat_shift, confidence=1-delta)

### Optional -- More complex models

The previous models assigns the same uncertainty interval to all predictions. We can alternatively use two more models that callibrate the confidence interval for each prediction based on their heuristic difficulty using k-nearest neighbours.

In [69]:
# Second model based on k-nn
sigmas_cal_knn = sigma_knn(X=X_cal, residuals=residuals_cal)
cr_norm_knn = ConformalRegressor()
cr_norm_knn.fit(residuals=residuals_cal, sigmas=sigmas_cal_knn)
sigmas_test_knn = sigma_knn(X=X_cal, residuals=residuals_cal, X_test=X_shift)
intervals_norm_knn = cr_norm_knn.predict(y_hat=y_hat_shift, 
                                        sigmas=sigmas_test_knn,
                                        y_min=0, y_max=1)

# Third model based on binning
bins_cal, bin_thresholds = binning(values=sigmas_cal_knn, bins=20)
cr_mond = ConformalRegressor()
cr_mond.fit(residuals=residuals_cal, bins=bins_cal)
bins_test = binning(values=sigmas_test_knn, bins=bin_thresholds)
intervals_mond = cr_mond.predict(y_hat=y_hat_shift, bins=bins_test, 
                                        y_min=0, y_max=1)

We can compute the coverage of the three models

In [70]:
coverages = []
mean_sizes = []
median_sizes = []

prediction_intervals = {
    "Std CR":intervals_std,
    "Norm CR knn":intervals_norm_knn,
    "Mond CR":intervals_mond,
}

for name in prediction_intervals.keys():
    intervals = prediction_intervals[name]
    coverages.append(np.sum([1 if (y_shift[i]>=intervals[i,0] and 
                                   y_shift[i]<=intervals[i,1]) else 0 
                            for i in range(len(y_shift))])/len(y_shift))
    mean_sizes.append((intervals[:,1]-intervals[:,0]).mean())
    median_sizes.append(np.median((intervals[:,1]-intervals[:,0])))

pred_int_df = pd.DataFrame({"Coverage":coverages, 
                            "Mean size":mean_sizes, 
                            "Median size":median_sizes}, 
                           index=list(prediction_intervals.keys()))

pred_int_df.loc["Mean"] = [pred_int_df["Coverage"].mean(), 
                           pred_int_df["Mean size"].mean(),
                           pred_int_df["Median size"].mean()]

display(pred_int_df.round(4))

,Coverage,Mean size,Median size
Std CR,0.9702,1.3413,1.3413
Norm CR knn,0.4735,0.7460,0.7964
Mond CR,0.4768,0.6657,0.6615
Mean,0.6402,0.9177,0.9331


## Auxiliary functions

In the previous section we have used pre-defined conformal models to compute the confidence intervals. We can also assume gaussian intervals derived from these residuals and reverse engineer the confidence intervals.

First we define function that computes the standard deviation from an interval and $1-\delta$.

In [71]:
def compute_sigma(delta, initial_interval, mean):
    """ Computes the standard deviation of a normal distribution such that the probability of the interval is 1-delta"""
    def cdf(sigma_):
        return 0.5*(erf((initial_interval[1]-mean)/(np.sqrt(2)*sigma_))-erf((initial_interval[0]-mean)/(np.sqrt(2)*sigma_))) - (1-delta)

    sigma = fsolve(cdf, (initial_interval[1]-initial_interval[0])/2)[0]
    return sigma

Then we define a function to compute the interval in which the (assumed normal) probability distribution is larger than $\alpha$. In other words, $\mathcal{C}_\alpha$.

In [72]:
def confidence_interval(mean, sigma, alpha, initial_interval_guess):
    """ We want to compute the conformal integral in which the probability density is larger than alpha. """
    def normal_distribution(x, mean, sigma):
        return np.exp(-((x-mean)**2)/(2*sigma**2))

    interval_alpha = fsolve(lambda x: normal_distribution(x, mean, sigma) - alpha, x0 = initial_interval_guess)
    return interval_alpha

Finally, we define a function to integrate the loss on a given interval.

In [73]:
def compute_loss(y_hat, y_gt, sigma, interval_alpha):
    """ Integrates the  quadratic loss over the interval defined by the probability density being larger than alpha."""
    def normal_loss(x, y, mean, sigma):
        return np.exp(-((x-mean)**2)/(2*sigma**2))*np.abs(x-y)

    normal_l = partial(normal_loss, y=y_gt, mean=y_hat, sigma = sigma)
    loss = quad(normal_l, interval_alpha[0], interval_alpha[1])[0]
    return loss

### Learn the Test

We also want to compute the p-value of the hypothesis $\mathcal{H}_\alpha$ that the risk $R(\alpha)\leq \lambda$ for a chosen $\lambda$.

In [74]:
lambda_ = 0.5

def pvalue(y_gt, y_hat, sigmas, lambda_):
    """ Computes the p-value of the hypothesis that the risk is lower than some \lambda."""

    pvalues = []

    # For each y
    alphas = np.linspace(0.01, 0.99, 100)

    for alpha in alphas:

        loss = 0
        for e in range(len(y_hat)):

            # Step 1: compute the confidence interval, then compute the loss
            interval_alpha = confidence_interval(mean = y_hat[e], sigma = sigmas[e], alpha = alpha, initial_interval_guess=[y_hat[e] - sigmas[e], y_hat[e]+sigmas[e]])
            loss += compute_loss(y_hat[e], y_gt[e], sigmas[e], interval_alpha)

        loss /= len(y_hat)

        # Step 1: Compute p-values
        p_value = np.exp(-2*len(y_hat)*(lambda_ - loss)**2)

        pvalues.append(p_value)

    # Step 2: Family-wise error correction
    reject, pvals_corrected, _, bonferroni_delta = multipletests(pvalues, delta, method = 'bonferroni')

    return alphas, pvals_corrected, reject

## Time-stratified loss metric

An alternative easy way is to check whether the loss of in the new dataset is larger than in the original model.

In [75]:
y_train_hat = random_forest_model.predict(X_train)
intervals_y_train = cr_std.predict(y_hat=y_hat_shift, confidence=1-delta)

In [76]:
def quick_loss(y_hat, y_train, alpha, intervals, delta):
    """ Computes the loss function for a given sample."""
    
    sigma = compute_sigma(delta, intervals, y_train)
    interval_alpha = confidence_interval(y_hat, sigma, alpha, [y_hat - sigma, y_hat + sigma])
    return compute_loss(y_hat, y_train, sigma, interval_alpha)

In [77]:
len(X_train), len(X_cal), len(X_shift)

(5202, 5000, 604)

In [78]:
losses = [] 
alpha = 0.5
for j in tqdm(range(10)):
    X_cal_subset = X_cal[j*500:(j+1)*500]
    y_cal_subset = y_cal[j*500:(j+1)*500]

    y_cal_hat_subset = random_forest_model.predict(X_cal_subset)
    intervals_y_train = cr_std.predict(y_hat=y_cal_hat_subset, confidence=1-delta)
    losses_ = []
    for i in range(len(y_cal_subset)):
        losses_.append(quick_loss(y_cal_hat_subset[i], y_cal_subset[i], alpha, intervals_y_train[i], delta))

    loss_train = np.mean(losses_)
    losses.append(loss_train)

y_shift_hat = random_forest_model.predict(X_shift)
intervals_shift = cr_std.predict(y_hat=y_shift_hat, confidence=1-delta)
loss_test = np.mean([quick_loss(y_shift_hat[i], y_shift[i], alpha, intervals_shift[i], delta) for i in range(len(y_shift))])

  0%|          | 0/10 [00:00<?, ?it/s]/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
 10%|█         | 1/10 [00:01<00:15,  1.78s/it]/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimiz

In [79]:
if loss_test > np.quantile(losses, 0.95):
    print('Distribution shift detected')
else:
    print('No distribution shift detected')
print(losses)
print(loss_test)

No distribution shift detected
[0.1666130828478795, 0.15923880730506504, 0.15492822982040014, 0.16412193287024435, 0.13726269959558407, 0.14259455340000202, 0.16336602189154723, 0.16674872534639962, 0.15982354885577132, 0.15472022755219164]
0.13736353618007696


## Split conformal

Basically we repeat the same analysis as above for some calibrated $\alpha$.

In [80]:
y_hat_cal = random_forest_model.predict(X_shift)
intervals = cr_std.predict(y_hat=y_hat_cal, confidence=1-delta)
sigmas = [compute_sigma(delta, intervals[e], y_hat_cal[e]) for e in range(len(y_hat_cal))]

alphas, pvals_corrected, reject = pvalue(y_shift, y_hat_cal, sigmas, lambda_)

# Alpha is the minium alpha that is significant
alphas = alphas[np.where(reject == True)][0]

losses = []

y_cal_hat = random_forest_model.predict(X_cal)
intervals_y_cal = cr_std.predict(y_hat=y_cal_hat, confidence=1-alpha)

for i in range(len(y_cal)):
    losses.append(quick_loss(y_cal_hat[i], y_cal[i], alpha, intervals_y_cal[i], delta))

loss_threshold = np.quantile(losses, 0.95)

y_shift_hat = random_forest_model.predict(X_shift)
intervals_shift = cr_std.predict(y_hat=y_shift_hat, confidence=1-delta)
above_threshold = []
for i in range(len(y_shift)):
    above_threshold.append(quick_loss(y_shift_hat[i], y_shift[i], alpha, intervals_shift[i], delta) > loss_threshold)
loss_test_check = np.sum(above_threshold)

# Implement binomial test
result = binomtest(loss_test_check, len(losses), 1-delta, alternative='greater')

if result.pvalue > epsilon:
    print('Distribution shift detected with pvalue {}'.format(result.pvalue))
else:
    print('No distribution shift detected with pvalue {}'.format(result.pvalue))

/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as 

Distribution shift detected with pvalue 1.0


## Inductive conformal predictors

We are going to compute the loss of exchanging one element of the training set with the shifted data. Then we will compute the corresponding p-values.

In [81]:
new_x = np.expand_dims(X_shift[0], axis = 0)
new_y = np.expand_dims(y_shift[0], axis = 0)

alpha = 1e-5
ncal = len(y_cal)

# First we fit a model to the data
random_forest_model = RandomForestRegressor(n_estimators=100)
random_forest_model.fit(X_train, y_train)

# We can now compute the residuals
y_hat_cal = random_forest_model.predict(X_cal)
residuals_cal = y_cal - y_hat_cal

# We can now fit a model to the residuals
cr_std = ConformalRegressor()
cr_std.fit(residuals=residuals_cal)

def find_lamdas(X, y, delta):
    lambdas_ = []
    sigmas = []
    intervals = []

    for yc, xc in zip(y, X):

        xc = xc.reshape(1, -1)

        # Compute predictions and confidence intervals
        yhc = random_forest_model.predict(xc)
        interval = cr_std.predict(y_hat=yhc, confidence=1-delta)[0]
        intervals.append(interval)

        # Compute the loss of the samples
        sigma = compute_sigma(delta = delta, initial_interval = interval, mean = yhc)
        sigmas.append(sigma)
        lambda_ = compute_loss(yhc, yc, sigma, interval)
        lambdas_.append(lambda_)

    return lambdas_, sigmas, intervals

# Compute the calibration lambdas
lambdas_i, sigmas, intervals = find_lamdas(X_cal, y_cal, delta = delta)

# Compute the test lambdas
lambdas_i_shift, sigmas_shift, intervals_shift = find_lamdas(X_shift, y_shift, delta = delta)

# We can further compute the percentile
p = np.percentile(lambdas_i, 100*delta)
#percentile = percentileofscore(lambdas_i, lambda_test)

We now want to compute the possible values of y such that the corresponding p-value is larger than $\epsilon$, $p^y \geq \epsilon$ for 
$$p^y := \frac{|\{ i \in 0,\ldots,n_{cal}| \lambda_i \leq \lambda_y\}|+1}{n_{cal}+1}\geq \epsilon.$$
First we compute how the p-value should be


In [82]:
np.quantile(lambdas_i, ((1-epsilon)*(ncal + 1) - 1)/len(lambdas_i))

0.5494613407605338

In [83]:
e = 0 # We can choose any sample from the shift dataset
epsilon = 0.05
lambda_y_upper_bound = np.quantile(lambdas_i, ((1-epsilon)*(len(lambdas_i) + 1) - 1)/len(lambdas_i))
print('The quantile {} of the calibration set is {}'.format(1-epsilon, lambda_y_upper_bound))
count = 0
for i in range(len(lambdas_i_shift)):
    if lambdas_i_shift[i] > lambda_y_upper_bound:
        print("We detect distribution shift, based on the p-value above")
        count += 1
    else:
        print("We do NOT detect distribution shift, based on the p-value above")

print("We detect distribution shift {} times out of {} samples".format(count, len(lambdas_i_shift)))

The quantile 0.95 of the calibration set is 0.5494613407605338
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, based on the p-value above
We do NOT detect distribution shift, base

We now want to find the values of y that satisfies such bound

In [84]:
compute_loss_partial = partial(compute_loss, y_gt = y_shift[e], sigma = sigmas[e], interval_alpha = intervals[e])

y_lower_bound = fsolve(lambda x: compute_loss_partial(x) - lambda_y_upper_bound, x0 = y_shift[e]-0.5)
y_upper_bound = fsolve(lambda x: compute_loss_partial(x) - lambda_y_upper_bound, x0 = y_shift[e]+0.5)

/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In practice we want to compute, for each value of x in the shifted dataset, the corresponding bounds on y

In [85]:
def compute_bounds(lambdas_i, y_shift_gt, lambda_shift, sigma, interval):
    
    lambda_y_upper_bound = np.quantile(lambdas_i, ((1-epsilon)*(ncal + 1) - 1)/len(lambdas_i))

    compute_loss_partial = partial(compute_loss, y_gt = y_shift_gt, sigma = sigma, interval_alpha = interval)

    bound0 = fsolve(lambda x: compute_loss_partial(x) - lambda_y_upper_bound, x0 = 0)
    for i in range (1000):
        x0 = np.random.uniform(-1, 1)
        bound1 = fsolve(lambda x: compute_loss_partial(x) - lambda_y_upper_bound, x0 = x0)
        if not np.allclose(bound0, bound1):
            break
    if i == 999:
        print('Warning: No bounds found for y_shift_gt = {}'.format(y_shift_gt))
        return None, None, None

    y_lower_bound, y_upper_bound = np.min([bound0, bound1]), np.max([bound0, bound1])

    return y_lower_bound, y_upper_bound, y_shift_gt < y_upper_bound and y_shift_gt > y_lower_bound

Now we can check whether the shifted dataset is in the confidence interval of the full conformal model with probability $1-\epsilon$.

In [86]:
for e in range(10):
    y_lower_bound, y_upper_bound, detect = compute_bounds(lambdas_i, y_shift[e], lambdas_i_shift[e], sigmas_shift[e], intervals_shift[e])
    print("For sample {}, is the prediction {} within bounds?: {} and the bounds are [{}, {}]".format(e, y_shift[e], detect, y_lower_bound, y_upper_bound))

/Users/pablo_1/opt/miniconda3/envs/conformal/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


For sample 0, is the prediction -0.14277065861237725 within bounds?: False and the bounds are [0.31101393098562424, 0.311626048896959]
For sample 1, is the prediction 0.022306074812668886 within bounds?: False and the bounds are [0.814922194860251, 0.8152449080494905]
For sample 2, is the prediction 0.06549014535697928 within bounds?: False and the bounds are [-0.5786658794066643, -0.5782390774477388]
For sample 3, is the prediction -0.016253317329608663 within bounds?: False and the bounds are [0.5583825592066956, 0.5589314344723375]
For sample 4, is the prediction 0.14884641180490288 within bounds?: False and the bounds are [-0.25862734719645514, -0.25826423397937326]
For sample 5, is the prediction -0.14662385165000236 within bounds?: False and the bounds are [0.6674730887970011, 0.667483819955549]
For sample 6, is the prediction -0.006522230018587094 within bounds?: False and the bounds are [0.429069517991568, 0.43019952262488875]
For sample 7, is the prediction -0.0523709734067422

In [87]:
outside = 0
for e in range(len(y_shift)):
    y_lower_bound, y_upper_bound, out = compute_bounds(lambdas_i, y_shift[e], lambdas_i_shift[e], sigmas_shift[e], intervals_shift[e])
    outside += out
print('The percentage of samples outside the bounds is: {}'.format(outside/len(y_shift)))

The percentage of samples outside the bounds is: 0.046357615894039736


In [88]:
outside

28

We can furthermore compute the p-value of the hypothesis that the shifted dataset is in the confidence interval of the full conformal model, using a binomial test.

In [89]:
test = binomtest(k = outside, n = len(y_shift), p = epsilon, alternative='greater')
print('p-value: ', test.pvalue)
print('Confidence interval: ', test.proportion_ci())

p-value:  0.6854028080562641
Confidence interval:  ConfidenceInterval(low=0.03314491437811541, high=1.0)
